In [ ]:
# Liner Regression

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
!pip install pytorch_lightning

In [ ]:
# import module 
import torch 

# To get the layers and losses for our model 
from torch import nn 
import pytorch_lightning as pl 

# To get the activation function for our model 
import torch.nn.functional as F 
import torch.nn as nn

# To get MNIST data and transforms 
from torchvision import datasets, transforms 

# To get the optimizer for our model 
from torch.optim import SGD 

# To get random_split to split training 
# data into training and validation data 
# and DataLoader to create dataloaders for train, 
# valid and test data to be returned 
# by our data module 
from torch.utils.data import random_split, DataLoader 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset


In [ ]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                    [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                    [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                    [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                    [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                    dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                  dtype='float32')

#inputs = torch.from_numpy(inputs)
#targets = torch.from_numpy(targets)

In [ ]:
len(inputs)

15

In [ ]:
mse_loss = nn.MSELoss(reduction = 'mean')


In [ ]:
class DataModulelinear(pl.LightningDataModule):
  def __init__(self):
    super().__init__()
    self.inputs = torch.from_numpy(inputs)
    self.targets = torch.from_numpy(targets)
    self.batch_size=5

  def setup(self,stage=None):
    self.train_ds = TensorDataset(self.inputs, self.targets)
    # self.train_data, self.valid_data = random_split(self.train_ds,[10, 5])

  def train_dataloader(self):
    return DataLoader(self.train_ds,batch_size = self.batch_size)
    
  # def val_dataloader():
  #   return DataLoader(self.valid_data, batch_size = self.batch_size) 

In [ ]:
class Regression(pl.LightningModule):
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(3, 2)
  def forward(self, x):
    x = self.fc1(x.float()) 
    return x
  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=1e-5)
  def training_step(self, train_batch, batch_idx):
    x, y = train_batch 
    logits = self.forward(x) 
    loss = mse_loss(logits, y) 
    return loss 
  # def validation_step(self, valid_batch, batch_idx): 
  #   x, y = valid_batch 
  #   logits = self.forward(x) 
  #   loss = mse_loss(logits, y) 
  #   return loss

In [ ]:
clf = Regression() 
mnist = DataModulelinear() 
trainer = pl.Trainer(max_epochs = 1) 
trainer.fit(clf, mnist) 

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 8     
--------------------------------
8         Trainable params
0         Non-trainable params
8         Total params


1

In [ ]:
trainer.save_checkpoint("linear.ckpt")

In [ ]:
!ls

lightning_logs	linear.ckpt  sample_data


In [ ]:
new_model = Regression.load_from_checkpoint(checkpoint_path="linear.ckpt")

In [ ]:
new_model

Regression(
  (fc1): Linear(in_features=3, out_features=2, bias=True)
)

In [ ]:
inputs1 = torch.from_numpy(inputs)
inputs1=inputs1[1]

In [ ]:
targets1 = torch.from_numpy(targets)
targets1=targets1[1]

In [ ]:
print(inputs1,targets1)

tensor([91., 88., 64.]) tensor([ 81., 101.])


In [ ]:
preds = new_model(inputs1.float())
preds

tensor([82.3907, 98.3927], grad_fn=<AddBackward0>)